In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

## Join ACS info to cbg shapefile by fips

In [6]:
# - Load census block groups
cbgs = gpd.read_file('/opt/laurelhurst4all/data/shapefiles/Census_Tracts_2010/Census_Tracts_2010.shp')
cbgs = cbgs[['GEOID10','geometry']]
cbgs.rename(columns={'GEOID10':'fips'}, inplace=True)
cbgs['fips'] = pd.to_numeric(cbgs['fips'])
cbgs['geometry'] = cbgs['geometry'].to_crs(epsg=4326)

In [7]:
cbgs.head()

,fips,geometry
0,53033002500,"POLYGON ((-122.29602 47.69023, -122.29604 47.6..."
1,53033002600,"POLYGON ((-122.30817 47.69031, -122.30793 47.6..."
2,53033005600,"POLYGON ((-122.39300 47.63956, -122.39300 47.6..."
3,53033006800,"POLYGON ((-122.35070 47.63994, -122.35071 47.6..."
4,53033006000,"POLYGON ((-122.34279 47.64320, -122.34267 47.6..."


In [ ]:
# - Load ACS 2016 info
acs = pd.read_csv('/opt/dssg-hot/data/acs/block_group_census_estimates_wide_original_bins_all_WA.csv',
                  usecols=['fips_code', 'households', 'mean_inc', 'med_age', 'med_inc', 'pc_income',
                              'population', 'race_nonhisp_asian', 'race_nonhisp_white', 'trans_carpool',
                              'trans_drivealone', 'trans_transit'])
acs.rename(columns={'fips_code':'fips'}, inplace=True)
cols = acs.columns.drop(['fips','population'])
#acs.dtypes
#acs.head()

In [ ]:
cbgs_acs = cbgs.merge(acs, on='fips')

## Load in shapefiles for drawing maps

### Load in roads shapefile

In [12]:
rds500k = gpd.read_file('/opt/laurelhurst4all/data/shapefiles/roads500k/sr500k_20201231.shp')
#print(rds500k.crs)
rds500k['geometry'] = rds500k['geometry'].to_crs(epsg=4326)
#print(rds500k.head())

majrds = rds500k[(rds500k['StateRoute']=='405')|(rds500k['StateRoute']=='005')
                |(rds500k['StateRoute']=='522')|(rds500k['StateRoute']=='520')
                |(rds500k['StateRoute']=='527')|(rds500k['StateRoute']=='90')]
#majrds.plot()

### Load in water shapefiles

In [16]:
water = gpd.read_file('/opt/laurelhurst4all/data/shapefiles/water/Open_water_for_King_County_and_portions_of_adjacent_counties___wtrbdy_area.shp')
#print(water.crs)
#print(water.head())

majwater = water[(water['NAME']=='Green Lake')|(water['NAME']=='Lake Union')
                |(water['NAME']=='Lake Meridian')|(water['NAME']=='Salmon Bay')
                |(water['NAME']=='Lake Washington')|(water['NAME']=='Lake Sammamish')
                |(water['NAME']=='Lake Youngs')|(water['NAME']=='Lake Stevens')
                |(water['NAME']=='Puget Sound')]
#majwater.plot()

### Load in redlining map

In [17]:
rl = gpd.read_file('/opt/laurelhurst4all/data/shapefiles/WASeattle1936/cartodb-query.shp')

In [18]:
print(rl.crs)
print(rl.head())

epsg:4326
   name holc_id holc_grade                                           geometry
0  None      A1          A  POLYGON ((-122.39875 47.69063, -122.39851 47.6...
1  None     A10          A  POLYGON ((-122.39747 47.58351, -122.39673 47.5...
2  None     A11          A  POLYGON ((-122.37809 47.58435, -122.37809 47.5...
3  None     A12          A  POLYGON ((-122.39761 47.55554, -122.39637 47.5...
4  None      A2          A  POLYGON ((-122.25787 47.67193, -122.25612 47.6...


In [ ]:
rl.plot()

### Load in zoning map

In [ ]:
zoning = gpd.read_file('/opt/laurelhurst4all/data/shapefiles/zoning/Land_Use_Zoning_Detail_with_Capacity_Assumptions.shp')
print(zoning.crs)
print(zoning.head())
zoning.plot()